In [ ]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

In [ ]:
ib =  get_connected('nse', 'live')

In [ ]:
# get the remaining quantities
dfrq = remqty_nse(ib)

dft = pd.read_pickle(fspath+'targets.pickle')

dft = dft.assign(remqty=dft.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
dft = dft[dft.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty <= 0)

In [ ]:
pmask = (dft.right == 'P') & (dft.undPrice-dft.strike-dft.expPrice > dft.Fall)

df_prisky = pd.merge(dft[pmask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom']], 
         pd.DataFrame((dft[pmask].undPrice-dft[pmask].strike-dft[pmask].expPrice)), on=dft[pmask].index).drop('key_0', 1)

cmask = (dft.right == 'C') & (dft.strike-dft.undPrice-dft.expPrice > dft.Rise)
df_crisky = pd.merge(dft[cmask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom']], 
         pd.DataFrame((dft[cmask].strike-dft[cmask].undPrice-dft[cmask].expPrice)), on=dft[cmask].index).drop('key_0', 1)

df_risky = pd.concat([df_prisky, df_crisky]).reset_index(drop=True)

df_risky = df_risky.rename(columns={0: 'FallRise'})

In [ ]:
df_risky

In [ ]:
dynamic(ib)